In [121]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

client = MongoClient()

In [2]:
#System-specific setup

db = client.forsaken
book_schemas_collection = db.rawSchemas
book_links_collection = db.rawLinks

In [3]:
def convert_fields_to_projection_without_id(fields):
    remove_id_projection = {"_id": 0}
    field_projection = {field: 1 for field in fields}
    projection = remove_id_projection | field_projection
    
    return projection

In [4]:
def get_entries(collection, projection):
    return list(collection.find(projection=projection))

In [5]:
book_schema_fields = ["title",
"era",
"compDate",
"dependence",
"categories"]

In [6]:
projection = convert_fields_to_projection_without_id(book_schema_fields)
book_schemas_list = get_entries(book_schemas_collection, projection)

In [7]:
book_schemas_dataframe = pd.DataFrame(book_schemas_list)

In [8]:
bs = book_schemas_dataframe

In [105]:
def regex_replace_in_field(pattern, replacement, field, dataframe):
    match_indexes = dataframe[field].str.match(pattern, na=False)
    matches = dataframe[match_indexes][field]
    fixed_matches = matches.str.replace(pattern, replacement, regex=True)
    
    dataframe.loc[match_indexes, field] = fixed_matches

In [126]:
from functools import partial

replace_date_with_capture = partial(regex_replace_in_field, field="compDate", dataframe=bs)

positive_capture = r"\1"
negative_capture = r"-\1"

replace_date_with_capture(r"(\d+)-\d+", positive_capture)
replace_date_with_capture(r"(\d+) BCE?", negative_capture)
replace_date_with_capture(r"ca?\.\s?(\d+)", positive_capture)

341      300
2796    1260
Name: compDate, dtype: object